In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from decimal import Decimal
from sklearn.model_selection import train_test_split

# --------------------------------------------------------------------------
# 0. 폴더/파일 설정
# --------------------------------------------------------------------------
# 기본 경로 설정
base_dir = r'C:\Users\idle9\Desktop\Naver MYBOX\AISLab\Dataset\AWID3_Dataset_CSV'

# 원본 CSV 데이터가 있는 폴더 (기본 경로 아래 CSV 폴더)
dataset_folder = os.path.join(base_dir, 'CSV')

# 공격 유형별 폴더와 파일 확장자
folders = {
    '1.Deauth': '.csv',
    # '2.Disas': '.csv',
    # '3.(Re)Assoc': '.csv',
    # '4.Rogue_AP': '.csv',
    # '5.Krack': '.csv',
    # '6.Kr00k': '.csv',
    # '7.SSH': '.csv',
    # '8.Botnet': '.csv',
    # '9.Malware': '.csv',
    # '10.SQL_Injection': '.csv',
    # '11.SSDP': '.csv',
    # '12.Evil_Twin': '.csv',
    # '13.Website_spoofing': '.csv'
}

# 전처리 결과를 저장할 기본 폴더 (기본 경로 아래 Preprocessed 폴더)
base_output_folder = os.path.join(base_dir, 'Preprocessed')
os.makedirs(base_output_folder, exist_ok=True)

# --------------------------------------------------------------------------
# 1. Label 매핑 (공격 이름 → 숫자)
# --------------------------------------------------------------------------
label_mapping = {
    'Normal': 0,
    'Deauth': 1,
    'Disas': 2,
    '(Re)Assoc': 3,
    'RogueAP': 4,
    'Krack': 5,
    'Kr00k': 6,
    'kr00k': 6,
    'SSH': 7,
    'Botnet': 8,
    'Malware': 9,
    'SQL_Injection': 10,
    'SSDP': 11,
    'SDDP': 11,
    'Evil_Twin': 12,
    'Website_spoofing': 13
}

# --------------------------------------------------------------------------
# 2. 시간 파싱 함수 (frame.time 변환)
# --------------------------------------------------------------------------
def parse_frame_time(time_str):
    """
    "Dec 18- 2020 17:42:58.482600000 GTB Standard Time" 형태를
    datetime으로 파싱한 후 timestamp(float)로 변환하여 반환.
    """
    if pd.isna(time_str):
        return np.nan

    # 불필요한 '-' 제거 (예: "Dec 18- 2020" → "Dec 18 2020")
    time_str = time_str.replace('-', '')
    # 끝에 붙는 ' GTB Standard Time' 제거
    time_str = time_str.replace(' GTB Standard Time', '')

    # 소수점 이하가 6자리를 초과하면 자르기 (datetime.strptime는 마이크로초까지만 지원)
    if '.' in time_str:
        parts = time_str.split(' ')
        for i, part in enumerate(parts):
            if '.' in part:
                main_sec, frac = part.split('.')
                frac = frac[:6]  # 6자리까지만 사용
                parts[i] = main_sec + '.' + frac
        time_str = ' '.join(parts)

    try:
        dt = datetime.strptime(time_str, '%b %d %Y %H:%M:%S.%f')
    except ValueError:
        try:
            dt = datetime.strptime(time_str, '%b %d %Y %H:%M:%S')
        except Exception as e:
            return np.nan
    return dt.timestamp()

# --------------------------------------------------------------------------
# 3. 문자열 → 숫자 변환 함수들
# --------------------------------------------------------------------------
def evaluate_string_formula(s):
    """
    "10-2" 같은 문자열 수식을 eval을 사용해 계산 후 숫자로 변환.
    실패하면 원본 반환.
    """
    try:
        return eval(s.replace('-', ' - '))
    except:
        return s

def hex_to_decimal(x):
    """
    16진수 문자열을 10진수로 변환.
    """
    try:
        return Decimal(int(x, 16))
    except:
        return x

def mac_to_int(mac_str):
    """
    MAC 주소("00:11:22:33:44:55") 형식을 정수로 변환.
    """
    if pd.isna(mac_str):
        return np.nan
    mac_str = str(mac_str)
    if mac_str.lower() == 'nan':
        return np.nan
    clean_mac_str = mac_str.replace(':', '').replace('-', '')
    try:
        return int(clean_mac_str, 16)
    except:
        return np.nan

# --------------------------------------------------------------------------
# 4. 폴더 단위 전처리 및 train/test 분리 함수 (공격 별 폴더 생성)
# --------------------------------------------------------------------------
def process_folder(folder_path, ext, base_output_folder, attack_name):
    """
    해당 폴더 내의 CSV 파일들을 모두 읽어 병합 후 전처리하고,
    train/test 분리하여 각 공격별 폴더 내에 CSV 파일로 저장한다.
    """
    # (1) 폴더 내 CSV 파일 읽어 병합
    data = pd.DataFrame()
    for filename in os.listdir(folder_path):
        if filename.endswith(ext):
            file_path = os.path.join(folder_path, filename)
            temp_df = pd.read_csv(file_path)
            data = pd.concat([data, temp_df], ignore_index=True)

    if data.empty:
        print(f"[{folder_path}] 폴더에 CSV 파일이 없습니다. 스킵합니다.")
        return

    # (2) 원본 DataFrame 복사 후 메모리 해제
    df = data.copy()
    del data

    # (3) frame.time → timestamp 변환 (존재할 경우)
    if 'frame.time' in df.columns:
        df['frame.time'] = df['frame.time'].apply(parse_frame_time)

    # (4) 단일값 열 제거
    drop_cols = [col for col in df.columns if df[col].nunique() == 1]
    df.drop(drop_cols, axis=1, inplace=True)

    # (5) 문자열 수식 평가 및 16진수, MAC 주소 변환
    for col in df.select_dtypes(include=['object']).columns:
        if col not in ['wlan.sa', 'wlan.da']:  # 예외 처리
            df[col] = df[col].apply(evaluate_string_formula)

    for col in df.select_dtypes(include=['object']).columns:
        if col not in ['wlan.sa', 'wlan.da']:  # 예외 처리
            df[col] = df[col].apply(hex_to_decimal)

    # MAC 주소 변환은 wlan.sa/wlan.da 제외하고 적용
    string_cols = df.select_dtypes(include=['object']).columns
    for col in string_cols:
        if col not in ['wlan.sa', 'wlan.da']:
            if ':' in str(df[col].iloc[0]) or '-' in str(df[col].iloc[0]):
                df[col] = df[col].apply(mac_to_int)

    # (6) 여전히 문자열인 열 제거하되, 'wlan.sa'와 'wlan.da'는 유지
    cols_to_keep = ['wlan.sa', 'wlan.da']
    numeric_cols = df.select_dtypes(exclude=['object']).columns.tolist()
    object_cols = df.select_dtypes(include=['object']).columns.tolist()
    object_cols = [col for col in object_cols if col in cols_to_keep]
    df = pd.concat([df[numeric_cols], df[object_cols]], axis=1)

    # (7) Label 매핑 (Label 컬럼이 있는 경우)
    if 'Label' in df.columns:
        df['label'] = df['Label'].map(label_mapping)
        # 필요에 따라 원본 Label 열 제거 가능
        # df.drop(['Label'], axis=1, inplace=True)

    # (8) train/test 분리 (stratify 적용)
    stratify_val = df['label'] if 'label' in df.columns else None
    train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=stratify_val)

    # (9) 공격 이름(폴더) 별 출력 폴더 생성
    attack_output_folder = os.path.join(base_output_folder, attack_name)
    os.makedirs(attack_output_folder, exist_ok=True)

    # (10) 각 폴더 내에 train, test CSV 파일 저장
    train_file = os.path.join(attack_output_folder, "train_preprocessed.csv")
    test_file  = os.path.join(attack_output_folder, "test_preprocessed.csv")
    train_df.to_csv(train_file, index=False)
    test_df.to_csv(test_file, index=False)
    print(f"[{attack_name}] 전처리 완료 및 저장: {attack_output_folder}")
    del df, train_df, test_df

# --------------------------------------------------------------------------
# 5. 각 공격 폴더 순회하며 전처리 및 train/test 분리 실행
# --------------------------------------------------------------------------
for attack_name, ext in folders.items():
    folder_path = os.path.join(dataset_folder, attack_name)
    process_folder(folder_path, ext, base_output_folder, attack_name)

print("=== 모든 공격별 전처리 및 train/test 분리 완료 ===")

C:\Users\idle9\AppData\Local\Temp\ipykernel_41524\706682368.py:214: DtypeWarning: Columns (20,21,23,24,43,44,45,48,50,51,62,65,71,73,77,78,79,80,82,84,85,87,90,91,92,93,98,99,100,101,102,103,104,107,108,109,110,111,118,119,120,122,132,137,138,139,141,143,145,146,147,148,149,150,151,153,155,157,158,159,160,161,162,163,164,166,167,168,184,185,189,190,192,193,194,196,197,203,204,205,209,210,211,212,213,215,217,218,219,247,248,249,250,251) have mixed types.Specify dtype option on import or set low_memory=False.
  process_folder(folder_path, ext, base_output_folder, attack_name)
C:\Users\idle9\AppData\Local\Temp\ipykernel_41524\706682368.py:214: DtypeWarning: Columns (71,73,77,78,79,80,82,84,85,90,91,92,93,98,99,100,101,102,103,104,107,108,109,110,111,118,166,168,183,184,185,190,192,193,194,195,197,200,201,202,203,204,205,206,209,210,211,212,213,215,217,218,219,249) have mixed types.Specify dtype option on import or set low_memory=False.
  process_folder(folder_path, ext, base_output_folder

[1.Deauth] 전처리 완료 및 저장: C:\Users\idle9\Desktop\Naver MYBOX\AISLab\Dataset\AWID3_Dataset_CSV\Preprocessed\1.Deauth
=== 모든 공격별 전처리 및 train/test 분리 완료 ===
